# Energy evaluation

Let's import protlego

In [5]:
import sys
sys.path.append("/agh/projects/noelia/github/")
from protlego.all import *

Let's fetch the same example we have been working on, the relationship between Rossmann and P-loop domains:

In [6]:
myhit= fetch_id('4413706')

Build all possible chimera-combinations from query and subject. As explained in detail in the "Building chimeras" section. The variable chimeras stores all the possible combinations,

In [7]:
a=Builder(myhit)
aln=a.get_alignment(myhit.query,myhit.no)
qPDB, sPDB = a.superimpose_structures(aln,partial_alignment=True)
chimeras=a.build_chimeras(partial_alignment=True)

2020-10-20 21:31:52,576 - protlego - INFO - Attempting to download domain d2dfda1 from the SCOP server
2020-10-20 21:31:53,795 - protlego - INFO - File downloaded as /tmp/tmpeurzi_g3.pdb
2020-10-20 21:31:53,795 - protlego - INFO - Attempting to download domain d1wa5a_ from the SCOP server
2020-10-20 21:31:54,972 - protlego - INFO - File downloaded as /tmp/tmpi7g2hjbp.pdb
2020-10-20 21:31:54,974 - protlego - INFO - Loading /tmp/tmpeurzi_g3.pdb as a chimera object
2020-10-20 21:31:55,096 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2020-10-20 21:31:55,099 - protlego - INFO - Loading /tmp/tmpi7g2hjbp.pdb as a chimera object
2020-10-20 21:31:55,160 - protlego - INFO - Attempting to download hhs file for d2dfda1 from the FUZZLE server
2020-10-20 21:31:55,294 - protlego - INFO - File downloaded as /tmp/tmpgo0876zj.hhs
2020-10-20 21:31:56,234 - protlego - INFO - 

### 1. Energy calculation using molecular mechanics (Amber and Charmm)

We can evaluate the potential energy of a chimera with the Charmm and Amber forcefieldds. Let's first take a chimera as an example.

In [8]:
chimera=Builder.mutate_nonstandards(chimeras['comb1_72'])

The built chimeras sometimes present internal clashes. For this reason, they tend to present very high potential energy scores. To compute a more realistic measure, we can minimize the structure using the minimize_potential_energy function. There's the possibility to run the minimization with GPU acceleration (provided there is a CUDA installation), and one can also choose to restraint the backbone or not.

In [9]:
minimize_potential_energy(chimera, 'amber', cuda=True, restraint_backbone=False)

2020-10-20 21:32:03,197 - propka - INFO - No pdbfile provided
/agh/projects/noelia/miniconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)
2020-10-20 21:32:14,428 - protlego - INFO - Energy before minimization -22060.479508084623 kcal/mol
2020-10-20 21:32:52,534 - protlego - INFO - Energy after minimization -122225.71913889165 kcal/mol


(Quantity(value=-122225.71913889165, unit=kilocalorie/mole),
 Protein with:
 Number of residues: 8656
 Number of atoms: 28076)

In [10]:
minimize_potential_energy(chimera, 'amber')

2020-10-20 21:33:04,752 - protlego - INFO - Energy before minimization -20780.603791067413 kcal/mol
2020-10-20 21:33:31,612 - protlego - INFO - Energy after minimization -122055.938399098 kcal/mol


(Quantity(value=-122055.938399098, unit=kilocalorie/mole),
 Protein with:
 Number of residues: 8656
 Number of atoms: 28076)

In [12]:
minimize_potential_energy(chimera, 'charmm')

2020-10-20 21:34:52,334 - protlego - INFO - Energy before minimization -22081.949545635354 kcal/mol
2020-10-20 21:35:15,114 - protlego - INFO - Energy after minimization -91719.16871389539 kcal/mol


(Quantity(value=-91719.16871389539, unit=kilocalorie/mole),
 Protein with:
 Number of residues: 7478
 Number of atoms: 24542)

As we have seen, the function minimize_potential_energy() outputs two variables, one with the potential energy, and another the resulting minimized chimera:
    

In [14]:
energy, min_chimera  = minimize_potential_energy(chimera, 'charmm', cuda=True, restraint_backbone=False)

2020-10-20 21:37:12,611 - protlego - INFO - Energy before minimization -21855.85011924912 kcal/mol
2020-10-20 21:37:36,874 - protlego - INFO - Energy after minimization -91703.95976417477 kcal/mol
